# Data Generator 

En este archivo estara la script encargada de generar informacion y de meterla en SQL de forma sencilla


In [49]:
import pymysql
from random_username.generate import generate_username
from random import randint
from datetime import date

In [12]:
# Conexion

conn = pymysql.connect(host='localhost', port=3306, user='root', passwd='', db='deuzum')
cut = conn.cursor()


En la cell de abajo se encuentran las funciones con las que podemos crear la información que queremos añadir a la base de datos. Las funciones que hemos creado para failitar este apartado son:

- *crear_info_usuarios_basica*: Crea cuentas de usuario aleatorias 
- *crear_info_cuenta_basica*: Crea cuentas de usuarios aleatorios
- *crear_info_historial_basica*: Crea transacciones aleatorias entre usuarios


El listado de parametros es el siguiente:




| Parametro                | Descripción                                      |
|--------------------------|--------------------------------------------------|
| c_size                   | Cantidad de elementos que el programa va a crear |
| min_[nombre de variable] | Valor mínimo que se le asignará a la variable    |
| max_[nombre de variable] | Valor máximo que se le asignará a la variable    |


In [81]:
def crear_info_usuarios_basica(c_size):
    add_SQL = "INSERT INTO `usuarios` (`user`, `pass`, `pregSeguridad`, `respuesta`) VALUES (%s, %s, '1', %s)"
    for i in range(0, c_size):
        #user
        tV_user, tV_pass, tV_res  = generate_username(3)
        cut.execute(add_SQL,(tV_user,tV_pass,tV_res))
        conn.commit()
        
def crear_info_cuentas_basica(c_size, min_num = 1, max_num = 1024, min_din = 1, max_din = 1024):
    add_SQL = "INSERT INTO `cuentas` (`user_id`, `numero`, `dinero`) VALUES (%s, %s, %s)"
    get_user_SQL = "SELECT `id` FROM `usuarios` ORDER BY RAND() LIMIT 1"
    for i in range(0, c_size):
        #user
        cut.execute(get_user_SQL)
        tV_user = cut.fetchone()
        tV_num, tV_din = randint(min_num,max_num), randint(min_din,max_din)
        
        cut.execute(add_SQL,(tV_user,tV_num,tV_din))
        conn.commit()

def crear_info_historial_basica(c_size, min_din = 1, max_din = 1024):
    add_SQL = "INSERT INTO `historial` (`emisor_id`, `receptor`, `cantidad`, `fecha`) VALUES (%s,%s,%s,%s)"
    get_user_SQL_hist = "SELECT `id` FROM `usuarios` ORDER BY RAND() LIMIT 2"
    todayS = (date.today()).strftime("%Y-%m-%d")
    for i in range(0, c_size):
        #user
        cut.execute(get_user_SQL_hist)
        tV_emi, tV_rece = cut.fetchmany(2)
        tV_din = randint(min_din,max_din)
        
        cut.execute(add_SQL,(tV_emi, tV_rece,tV_din,todayS))
        conn.commit()



In [82]:
#crear_info_usuarios_basica(10)
#conn.close()
crear_info_cuentas_basica(1)

In [47]:
maxLen, maxLenp = 0,0
for j in range(0,100):
        
    maxLen = max(maxLen, len(generate_username(1)[0]))
    if(maxLen> maxLenp):
        print(str(maxLen) + " at " + str(j))
        maxLenp = maxLen
print(str(maxLen))

17 at 0
17
